# Ekstraksi dan Pengolahan Data Kepemilikan Tanah

## 1. Menyiapkan coding environment, termasuk menginstall dan/atau memperbarui library

In [1]:
# Menginstall library-library
%pip install pandas
%pip install pytesseract
%pip install pdf2image
%pip install pillow
%pip install numpy
%pip install matplotlib
%pip install tensorflow
%pip install opencv-python


# Mengimport library-library untuk digunakan
import cv2
import numpy as np
import os
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
from matplotlib import pyplot as plt
import re

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Loading the preprocessed template CSV file
template = pd.read_csv('DATA C-DESA NOBOREJO preprocessed.csv')

##### Konversi PDF ke PNG 

In [3]:
# Initialize a new empty dataframe with the same columns as the template
df = pd.DataFrame(columns=template.columns)
df.head()

# Make sure the PDF file is in the same directory as this script
output_folder = 'Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Save CSV to the output directory
output_dir = 'Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff'
output_file = 'output.csv'
df.to_csv(os.path.join(output_dir, output_file), index=False)

In [4]:
# Create subdirectory to hold images inside the Berkas Files subdirectory
output_folder = 'Berkas Files/images of berkas 500-600'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert PDF to images
pdf_path = 'Berkas Files/berkas 500-600.pdf'
images = convert_from_path(pdf_path)

# Save each image to the output folder
for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f'halaman_{i+1}.png')
    image.save(image_path, 'PNG')

# Display the paths of the saved images
image_paths = [os.path.join(output_folder, f'halaman_{i+1}.png') for i in range(len(images))]
print("Images saved at:")
for path in image_paths:
    print(path)

Images saved at:
Berkas Files/images of berkas 500-600\halaman_1.png
Berkas Files/images of berkas 500-600\halaman_2.png
Berkas Files/images of berkas 500-600\halaman_3.png
Berkas Files/images of berkas 500-600\halaman_4.png
Berkas Files/images of berkas 500-600\halaman_5.png
Berkas Files/images of berkas 500-600\halaman_6.png
Berkas Files/images of berkas 500-600\halaman_7.png
Berkas Files/images of berkas 500-600\halaman_8.png
Berkas Files/images of berkas 500-600\halaman_9.png
Berkas Files/images of berkas 500-600\halaman_10.png
Berkas Files/images of berkas 500-600\halaman_11.png
Berkas Files/images of berkas 500-600\halaman_12.png
Berkas Files/images of berkas 500-600\halaman_13.png
Berkas Files/images of berkas 500-600\halaman_14.png
Berkas Files/images of berkas 500-600\halaman_15.png
Berkas Files/images of berkas 500-600\halaman_16.png
Berkas Files/images of berkas 500-600\halaman_17.png
Berkas Files/images of berkas 500-600\halaman_18.png
Berkas Files/images of berkas 500-600\

In [5]:
# Load the image
image_path = 'Berkas Files/images of berkas 500-600/halaman_1.png'
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

if image is None:
    raise ValueError(f"Image not loaded. Check if the path is correct: {image_path}")

#### !!! TESTING "berkas 500-520 EXAMPLE 2.pdf !!!

##### Image preprocessing attempt 1, **failed**

Tried using: 

- **Tesseract OCR**: Used for extracting text from images.
- **Image Preprocessing**:
  - **Thresholding**: Applied adaptive thresholding to convert the image to a binary format.
  - **Noise Removal**: Used morphological operations to remove noise from the image.
  - **Contour Detection**: Detected contours to identify table structures in the image.
- **Table Line Detection**: Highlighted table lines to improve OCR accuracy.

Despite these efforts, we encountered challenges due to the poor quality of the scanned images and the handwritten text, which resulted in limited meaningful progress.

In [6]:
# Step 1: Apply refined adaptive thresholding with thicker lines
'''
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale (just in case)
adaptive_binary = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 1001, 20  # Increased block size and constant
)

# Save the output for this step
step_1_path = os.path.join(output_dir, 'Step 1 - Thickened Adaptive Thresholded Image.png')
cv2.imwrite(step_1_path, adaptive_binary)

print(f"Step 1 completed. Thickened adaptive thresholded image saved at: {step_1_path}")
'''

'\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale (just in case)\nadaptive_binary = cv2.adaptiveThreshold(\n    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 1001, 20  # Increased block size and constant\n)\n\n# Save the output for this step\nstep_1_path = os.path.join(output_dir, \'Step 1 - Thickened Adaptive Thresholded Image.png\')\ncv2.imwrite(step_1_path, adaptive_binary)\n\nprint(f"Step 1 completed. Thickened adaptive thresholded image saved at: {step_1_path}")\n'

In [7]:
# Step 2: Remove unwanted hand region
'''
# Adjust the bounding box to cover the hand
hand_region = (0, adaptive_binary.shape[0] - 800, 600, 1000)  # x, y, width, height
x, y, w, h = hand_region

# Mask the hand by painting the region white
hand_removed = adaptive_binary.copy()
cv2.rectangle(hand_removed, (x, y), (x + w, y + h), (255, 255, 255), -1)

# Save the cleaned image
step_2_path = os.path.join(output_dir, 'Step 2 - Hand Removed.png')
cv2.imwrite(step_2_path, hand_removed)

print(f"Step 2 completed. Hand removed image saved at: {step_2_path}")
'''

'\n# Adjust the bounding box to cover the hand\nhand_region = (0, adaptive_binary.shape[0] - 800, 600, 1000)  # x, y, width, height\nx, y, w, h = hand_region\n\n# Mask the hand by painting the region white\nhand_removed = adaptive_binary.copy()\ncv2.rectangle(hand_removed, (x, y), (x + w, y + h), (255, 255, 255), -1)\n\n# Save the cleaned image\nstep_2_path = os.path.join(output_dir, \'Step 2 - Hand Removed.png\')\ncv2.imwrite(step_2_path, hand_removed)\n\nprint(f"Step 2 completed. Hand removed image saved at: {step_2_path}")\n'

In [8]:
# Step 3: Highlight table lines in green (retain original background)
'''
try:
    # Detect edges using Canny
    edges = cv2.Canny(hand_removed, 500, 1500)  # Adjust thresholds as needed (low_threshold, high_threshold)

    # Dilate the edges to strengthen lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # Adjust kernel size for line thickness
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)

    # Filter detected edges based on geometry (long horizontal/vertical lines)
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(edges)  # Create a blank mask
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else 0
        # Keep only long horizontal or vertical lines
        if (w > 50 and h < 10) or (h > 50 and w < 10):  # Adjust thresholds as needed
            cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)

    # Combine mask with dilated edges
    filtered_lines = cv2.bitwise_and(dilated_edges, mask)

    # Convert grayscale image to BGR for color overlay
    table_lines_colored = cv2.merge([hand_removed, hand_removed, hand_removed])  # Grayscale to BGR

    # Overlay green color on detected table lines
    table_lines_colored[filtered_lines > 0] = [0, 255, 0]  # Green (0, 255, 0)

    # Save the resulting image
    step_3_path = os.path.join(output_dir, 'Step 3 - Filtered Green Table Lines.png')
    cv2.imwrite(step_3_path, table_lines_colored)

    print(f"Step 3 completed. Filtered green table lines image saved at: {step_3_path}")

except Exception as e:
    print(f"An error occurred in Step 3: {e}")
'''

'\ntry:\n    # Detect edges using Canny\n    edges = cv2.Canny(hand_removed, 500, 1500)  # Adjust thresholds as needed (low_threshold, high_threshold)\n\n    # Dilate the edges to strengthen lines\n    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # Adjust kernel size for line thickness\n    dilated_edges = cv2.dilate(edges, kernel, iterations=1)\n\n    # Filter detected edges based on geometry (long horizontal/vertical lines)\n    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\n    mask = np.zeros_like(edges)  # Create a blank mask\n    for contour in contours:\n        x, y, w, h = cv2.boundingRect(contour)\n        aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else 0\n        # Keep only long horizontal or vertical lines\n        if (w > 50 and h < 10) or (h > 50 and w < 10):  # Adjust thresholds as needed\n            cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)\n\n    # Combine mask with dilated e

##### Image preprocessing attempt 2

In [9]:
# Define input path and output folder
image_path = 'Berkas Files/images of berkas 500-600/halaman_1.png'
output_folder = 'Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

Step 1: Image Preprocessing

In [10]:
# Step 1.1. Convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Save the grayscale image
output_path = os.path.join(output_folder, 'halaman_1_gray.png')
cv2.imwrite(output_path, gray_image)

print(f"Grayscale image saved at: {output_path}")

Grayscale image saved at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff\halaman_1_gray.png


In [11]:
# Step 1.2. Enhance contrast using CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced_image = clahe.apply(gray_image)

# Save the enhanced contrast image
output_path = os.path.join(output_folder, 'halaman_1_contrast.png')
cv2.imwrite(output_path, enhanced_image)

print(f"Enhanced contrast image saved at: {output_path}")

Enhanced contrast image saved at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff\halaman_1_contrast.png


In [12]:
# Step 1.3. Remove noise using median blurring
denoised_image = cv2.medianBlur(enhanced_image, 5)

# Save the denoised image
output_path = os.path.join(output_folder, 'halaman_1_denoised.png')
cv2.imwrite(output_path, denoised_image)

print(f"Denoised image saved at: {output_path}")

Denoised image saved at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff\halaman_1_denoised.png


In [13]:
# Step 1.4. Normalize dimensions
# Define the target dimensions
target_width, target_height = 1024, 1024

# Resize the image while preserving the aspect ratio
h, w = denoised_image.shape
scaling_factor = min(target_width / w, target_height / h)
new_width = int(w * scaling_factor)
new_height = int(h * scaling_factor)

# Resize the image
resized_image = cv2.resize(denoised_image, (new_width, new_height), interpolation=cv2.INTER_AREA)

# Pad the image to fit the target dimensions
delta_w = target_width - new_width
delta_h = target_height - new_height
top, bottom = delta_h // 2, delta_h - (delta_h // 2)
left, right = delta_w // 2, delta_w - (delta_w // 2)
normalized_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)

# Save the normalized image
output_path = os.path.join(output_folder, 'halaman_1_normalized.png')
cv2.imwrite(output_path, normalized_image)

print(f"Normalized image saved at: {output_path}")

Normalized image saved at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff\halaman_1_normalized.png


Step 2: Removing the table

In [14]:
# Step 2.1: Detect Table Structures

# Threshold the image to binary
_, binary = cv2.threshold(normalized_image, 200, 255, cv2.THRESH_BINARY_INV)

# Detect horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
horizontal_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel)

# Detect vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 50))
vertical_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, vertical_kernel)

# Combine horizontal and vertical lines
table_structure = cv2.add(horizontal_lines, vertical_lines)

# Optional: Dilate to merge gaps in table lines
table_structure = cv2.dilate(table_structure, np.ones((3, 3), np.uint8))

# Save the resulting table structure image
output_path = 'Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_table_structure.png'
cv2.imwrite(output_path, table_structure)

# Print confirmation and show the saved output
print(f"Table structure image saved at: {output_path}")


Table structure image saved at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_table_structure.png


In [15]:
# Step 2.2: Remove Table Structure to Extract Text

# Normalize the table structure to binary (0 and 255)
_, table_structure_binary = cv2.threshold(table_structure, 127, 255, cv2.THRESH_BINARY)

# Create a mask where table lines are present
table_lines_mask = table_structure_binary

# Replace table lines in the normalized image with white
text_only_image = cv2.add(normalized_image, table_lines_mask)

# Save the resulting text-only image
output_path = 'Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_text_only.png'
cv2.imwrite(output_path, text_only_image)

# Print confirmation and save output
print(f"Text-only image saved at: {output_path}")

Text-only image saved at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_text_only.png


Step 3: Performing OCR

In [16]:
# Step 3.1: Apply OCR to Extract Text
# Specify OCR configurations
custom_config = r'--psm 13 -c tessedit_char_whitelist="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,() "'

# Apply OCR on the text-only image
ocr_result = pytesseract.image_to_string(text_only_image, lang='eng', config='custom_config')

# Save OCR output to a text file
output_path = 'Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_output_sparse.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(ocr_result)

# Print confirmation and OCR result
print(f"OCR result (sparse) saved at: {output_path}")
print("OCR output (sparse):")
print(ocr_result)

OCR result (sparse) saved at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_output_sparse.txt
OCR output (sparse):
NAMA WAJIB IPEDA
RING
akan TANAH KE! _|
Menara daftar “raindan
Nomor eincan Sebab dan
er Seer ‘anenl
— Lees mith i Lane mie pee perobahan
Shey ;
Gos ae) at

o_4es | a
tive sie he
“in 7

4

ilddataus




In [17]:
# Clean up the OCR result
cleaned_ocr = re.sub(r'[^a-zA-Z0-9\s.,]', '', ocr_result)  # Keep only alphanumeric and basic punctuation
cleaned_ocr = re.sub(r'\s+', ' ', cleaned_ocr).strip()  # Remove extra whitespace and trim

# Save cleaned OCR output to a text file
output_path_cleaned = 'Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_cleaned.txt'
with open(output_path_cleaned, 'w', encoding='utf-8') as f:
    f.write(cleaned_ocr)

# Print confirmation and cleaned output
print(f"Cleaned OCR result saved at: {output_path_cleaned}")
print("Cleaned OCR output:")
print(cleaned_ocr)

Cleaned OCR result saved at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_cleaned.txt
Cleaned OCR output:
NAMA WAJIB IPEDA RING akan TANAH KE Menara daftar raindan Nomor eincan Sebab dan er Seer anenl Lees mith i Lane mie pee perobahan Shey Gos ae at o4es a tive sie he in 7 4 ilddataus


In [18]:
# Step 3.3: Save OCR Output with Proper Header Placement

# Define page identifier and header
header = ["OCR Output"]
page_id = "page_500"

# Split cleaned OCR into individual words
words = [page_id] + cleaned_ocr.split()

# Create a DataFrame with words as a single row
df = pd.DataFrame([words])

# Update the header manually
df.columns = header + ["" for _ in range(len(words) - 1)]

# Save the DataFrame to a CSV
csv_output_path = 'Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_paginated_final_header.csv'
df.to_csv(csv_output_path, index=False, header=False)

# Print confirmation and preview
print(f"Final paginated OCR result saved to CSV at: {csv_output_path}")
print("CSV preview:")
print(df.head())

Final paginated OCR result saved to CSV at: Berkas Files/images of berkas 500-600/Preprocessed halaman 1 stuff/halaman_1_ocr_paginated_final_header.csv
CSV preview:
  OCR Output                                                             ...  \
0   page_500  NAMA  WAJIB  IPEDA  RING  akan  TANAH  KE  Menara  daftar  ...   

                                                    
0  at  o4es  a  tive  sie  he  in  7  4  ilddataus  

[1 rows x 38 columns]
